# 1. Import Libraries

In [ ]:
import sys, os, io, gc
import numpy as np
import pandas as pd
from pathlib import Path
from datetime import timedelta
from functools import reduce
from tqdm import tqdm
import optuna
import mlb

from lightgbm import LGBMRegressor
import optuna.integration.lightgbm as lgbm
from sklearn.metrics import mean_absolute_error

import warnings
warnings.filterwarnings("ignore")
print("Setup Complete")

In [ ]:
def reduce_memory_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2
    
    for col in df.columns:
        col_type = df[col].dtypes
        
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

def percentage_missing_values(df):
    missing_values_count = df.isnull().sum()
    total_cells = np.product(df.shape)
    total_missing = missing_values_count.sum()
    print ("Percentage of Missing Data = ",(total_missing/total_cells) * 100,"%")

# 2. Data Processing

In [ ]:
rosters_cols = ['playerId', 'teamId', 'status', 'date']

standings_cols = ['teamId', 'wins', 'losses', 'lastTenWins', 'lastTenLosses', 'date']

scores_cols =['playerId', 'battingOrder', 'gamesPlayedBatting', 'flyOuts',
              'groundOuts', 'runsScored', 'doubles', 'triples', 'homeRuns',
              'strikeOuts', 'baseOnBalls', 'intentionalWalks', 'hits',
              'hitByPitch', 'atBats', 'caughtStealing', 'stolenBases',
              'groundIntoDoublePlay', 'groundIntoTriplePlay', 'plateAppearances',
              'totalBases', 'rbi', 'leftOnBase', 'sacBunts', 'sacFlies',
              'catchersInterference', 'pickoffs', 'gamesPlayedPitching',
              'gamesStartedPitching', 'completeGamesPitching', 'shutoutsPitching',
              'winsPitching', 'lossesPitching', 'flyOutsPitching', 'airOutsPitching',
              'groundOutsPitching','runsPitching', 'doublesPitching', 'triplesPitching',
              'homeRunsPitching', 'strikeOutsPitching', 'baseOnBallsPitching',
              'intentionalWalksPitching', 'hitsPitching', 'hitByPitchPitching',
              'atBatsPitching', 'caughtStealingPitching', 'stolenBasesPitching',
              'inningsPitched', 'saveOpportunities', 'earnedRuns', 'battersFaced',
              'outsPitching', 'pitchesThrown', 'balls', 'strikes', 'hitBatsmen',
              'balks', 'wildPitches', 'pickoffsPitching', 'rbiPitching', 'date',
              'gamesFinishedPitching', 'inheritedRunners', 'inheritedRunnersScored',
              'catchersInterferencePitching', 'sacBuntsPitching', 'sacFliesPitching',
              'saves', 'holds', 'blownSaves', 'assists', 'putOuts', 'errors', 'chances']

feature_cols = ['label_playerId', 'label_primaryPositionName', 'label_teamId',
                'label_status', 'playerId', 'battingOrder', 'gamesPlayedBatting',
                'flyOuts', 'groundOuts', 'runsScored', 'doubles', 'triples',
                'homeRuns', 'strikeOuts', 'baseOnBalls', 'intentionalWalks',
                'hits', 'hitByPitch', 'atBats', 'caughtStealing', 'stolenBases',
                'groundIntoDoublePlay', 'groundIntoTriplePlay', 'plateAppearances',
                'totalBases', 'rbi', 'leftOnBase', 'sacBunts', 'sacFlies',
                'catchersInterference', 'pickoffs', 'gamesPlayedPitching',
                'gamesStartedPitching', 'completeGamesPitching', 'shutoutsPitching',
                'winsPitching', 'lossesPitching', 'flyOutsPitching', 'airOutsPitching',
                'groundOutsPitching', 'runsPitching', 'doublesPitching',
                'triplesPitching', 'homeRunsPitching', 'strikeOutsPitching',
                'baseOnBallsPitching', 'intentionalWalksPitching', 'hitsPitching',
                'hitByPitchPitching', 'atBatsPitching', 'caughtStealingPitching',
                'stolenBasesPitching', 'inningsPitched', 'saveOpportunities',
                'earnedRuns', 'battersFaced', 'outsPitching', 'pitchesThrown',
                'balls', 'strikes', 'hitBatsmen', 'balks', 'wildPitches',
                'pickoffsPitching', 'rbiPitching', 'gamesFinishedPitching',
                'inheritedRunners', 'inheritedRunnersScored',
                'catchersInterferencePitching', 'sacBuntsPitching', 'sacFliesPitching',
                'saves', 'holds', 'blownSaves', 'assists', 'putOuts', 'errors',
                'chances', 'target1_mean', 'target1_median', 'target1_std',
                'target1_min', 'target1_max', 'target1_prob', 'target2_mean',
                'target2_median', 'target2_std', 'target2_min', 'target2_max',
                'target2_prob', 'target3_mean', 'target3_median', 'target3_std',
                'target3_min', 'target3_max', 'target3_prob', 'target4_mean',
                'target4_median', 'target4_std', 'target4_min', 'target4_max',
                'target4_prob', 'awardId_count', 'playernumberOfFollowers',
                'teamnumberOfFollowers', 'label_leagueId', 'label_divisionId',
                'wins', 'losses', 'lastTenWins', 'lastTenLosses']

In [ ]:
# Source: https://www.kaggle.com/c/mlb-player-digital-engagement-forecasting/data
players = pd.read_csv('../input/mlb-player-digital-engagement-forecasting/players.csv', usecols = ['playerId', 'primaryPositionName'])
players = reduce_memory_usage(players)

teams = pd.read_csv('../input/mlb-player-digital-engagement-forecasting/teams.csv', usecols = ['id', 'leagueId', 'divisionId'])
teams = teams.rename(columns = {'id':'teamId'})
teams = reduce_memory_usage(teams)

In [ ]:
# Source: https://www.kaggle.com/columbia2131/mlb-pdef-train-dataset
rosters = pd.read_csv('../input/mlb-pdef-train-dataset/rosters_train.csv', usecols = rosters_cols)
rosters = reduce_memory_usage(rosters)

targets = pd.read_csv('../input/mlb-pdef-train-dataset/nextDayPlayerEngagement_train.csv', usecols = ['playerId', 'target1', 'target2', 'target3', 'target4', 'date'])
targets = reduce_memory_usage(targets)

scores = pd.read_csv('../input/mlb-pdef-train-dataset/playerBoxScores_train.csv', usecols = scores_cols)
scores = scores.groupby(['playerId', 'date']).sum().reset_index()
scores = reduce_memory_usage(scores)

awards = pd.read_csv('../input/mlb-pdef-train-dataset/awards_train.csv', usecols = ['date', 'playerId', 'awardId'])
awards_count = awards[['playerId', 'awardId']].groupby('playerId').count().reset_index()
awards_count = awards_count.rename(columns = {'awardId':'awardId_count'})
awards_count = reduce_memory_usage(awards_count)

playerTwitterFollowers = pd.read_csv('../input/mlb-pdef-train-dataset/playerTwitterFollowers_train.csv', usecols = ['playerId', 'numberOfFollowers'])
playerTwitterFollowers = playerTwitterFollowers.groupby('playerId').sum().reset_index()
playerTwitterFollowers = playerTwitterFollowers.rename(columns = {'numberOfFollowers':'playernumberOfFollowers'})
playerTwitterFollowers = reduce_memory_usage(playerTwitterFollowers)

teamTwitterFollowers = pd.read_csv('../input/mlb-pdef-train-dataset/teamTwitterFollowers_train.csv', usecols = ['teamId', 'numberOfFollowers'])
teamTwitterFollowers = teamTwitterFollowers.groupby('teamId').sum().reset_index()
teamTwitterFollowers = teamTwitterFollowers.rename(columns = {'numberOfFollowers':'teamnumberOfFollowers'})
teamTwitterFollowers = reduce_memory_usage(teamTwitterFollowers)

standings = pd.read_csv('../input/mlb-pdef-train-dataset/standings_train.csv', usecols = standings_cols)
standings = reduce_memory_usage(standings)

In [ ]:
# Source: https://www.kaggle.com/mlconsult/player-target-stats
player_target_stats = pd.read_csv('../input/player-target-stats/player_target_stats.csv')
data_names=player_target_stats.columns.values.tolist()

In [ ]:
train = targets.copy()[['playerId', 'target1', 'target2', 'target3', 'target4', 'date']]
train = train.merge(players, on=['playerId'], how='left')
train = train.merge(rosters, on=['playerId', 'date'], how='left')
train = train.merge(scores, on=['playerId', 'date'], how='left')
train = train.merge(player_target_stats, how='inner', on= "playerId")
train = train.merge(teams, on = 'teamId', how='left')
train = train.merge(awards_count, on = 'playerId', how = 'left')
train['awardId_count'] = train['awardId_count'].fillna(0)
train = train.merge(playerTwitterFollowers, how = 'left', on = 'playerId')
train = train.merge(teamTwitterFollowers, how = 'left', on = 'teamId')
train = train.merge(standings, how = 'left', on = ['teamId', 'date'])

In [ ]:
# Label Encoding
player2num = {c: i for i, c in enumerate(train['playerId'].unique())}
position2num = {c: i for i, c in enumerate(train['primaryPositionName'].unique())}
teamid2num = {c: i for i, c in enumerate(train['teamId'].unique())}
status2num = {c: i for i, c in enumerate(train['status'].unique())}
leagueId2num = {c: i for i, c in enumerate(train['leagueId'].unique())}
divisionId2num = {c: i for i, c in enumerate(train['divisionId'].unique())}

train['label_playerId'] = train['playerId'].map(player2num)
train['label_primaryPositionName'] = train['primaryPositionName'].map(position2num)
train['label_teamId'] = train['teamId'].map(teamid2num)
train['label_status'] = train['status'].map(status2num)
train['label_leagueId'] = train['leagueId'].map(leagueId2num)
train['label_divisionId'] = train['divisionId'].map(divisionId2num)

In [ ]:
percentage_missing_values(train)

In [ ]:
train_X = train[feature_cols]
train_y = train[['target1', 'target2', 'target3', 'target4']]

_index = (train['date'] < 20210401)
x_train = train_X.loc[_index].reset_index(drop=True)
y_train = train_y.loc[_index].reset_index(drop=True)
x_valid = train_X.loc[~_index].reset_index(drop=True)
y_valid = train_y.loc[~_index].reset_index(drop=True)

# 4. Hyperparameter Tuning

In [ ]:
params = {
 'objective':'mae',
 'reg_alpha': 0.1,
 'reg_lambda': 0.1, 
 'n_estimators': 100000,
 'learning_rate': 0.1,
 'random_state': 42,
}

In [ ]:
def lgbm_optuna(x_train, y_train, x_valid, y_valid, params: dict=None, verbose=100):
        
    oof_pred = np.zeros(len(y_valid), dtype=np.float32)    
    
    trains = lgbm.Dataset(x_train, y_train)
    valids = lgbm.Dataset(x_valid, y_valid)
    
    model = lgbm.train(
        params, 
        trains,
        valid_sets = valids,
        num_boost_round = 1,
        verbose_eval = False,
        early_stopping_rounds = 1
    )
    
    best_params = model.params
    
    oof_pred = model.predict(x_valid)
    score = mean_absolute_error(oof_pred, y_valid)
    print('MAE Score:', score)
    
    return oof_pred, model, score, best_params

In [ ]:
"""
oof1, model1, score1, best_params1 = lgbm_optuna(
        x_train, y_train['target1'],
        x_valid, y_valid['target1'],
        params
)

oof2, model2, score2, best_params2 = lgbm_optuna(
        x_train, y_train['target2'],
        x_valid, y_valid['target2'],
        params
)

oof3, model3, score3, best_params3 = lgbm_optuna(
        x_train, y_train['target3'],
        x_valid, y_valid['target3'],
        params
)

oof4, model4, score4, best_params4 = lgbm_optuna(
        x_train, y_train['target4'],
        x_valid, y_valid['target4'],
        params
)
"""

In [ ]:
params1 = {'objective': 'mae', 'metric': 'l1', 'feature_pre_filter': False, 'lambda_l1': 3.485822021802935e-08, 'lambda_l2': 4.230468117096112e-06, 'num_leaves': 253, 'feature_fraction': 0.8, 'bagging_fraction': 0.550250698524785, 'bagging_freq': 1, 'min_child_samples': 20, 'num_iterations': 10000, 'early_stopping_round': 100}
params2 = {'objective': 'mae', 'metric': 'l1', 'feature_pre_filter': False, 'lambda_l1': 3.731605225849285, 'lambda_l2': 0.02803980626777797, 'num_leaves': 8, 'feature_fraction': 0.5, 'bagging_fraction': 0.5262728428461787, 'bagging_freq': 3, 'min_child_samples': 20, 'num_iterations': 10000, 'early_stopping_round': 100}
params3 = {'objective': 'mae', 'metric': 'l1', 'feature_pre_filter': False, 'lambda_l1': 7.654830305013684, 'lambda_l2': 4.14748542765967e-07, 'num_leaves': 252, 'feature_fraction': 0.7200000000000001, 'bagging_fraction': 1.0, 'bagging_freq': 0, 'min_child_samples': 20, 'num_iterations': 10000, 'early_stopping_round': 100}
params4 = {'objective': 'mae', 'metric': 'l1', 'feature_pre_filter': False, 'lambda_l1': 9.486880706514734e-08, 'lambda_l2': 0.005143767850872896, 'num_leaves': 246, 'feature_fraction': 0.5479999999999999, 'bagging_fraction': 0.5238463354446826, 'bagging_freq': 5, 'min_child_samples': 20, 'num_iterations': 10000, 'early_stopping_round': 100}

# 5. LightGBM

In [ ]:
def lgbm(x_train, y_train, x_valid, y_valid, params: dict=None):
    oof_pred = np.zeros(len(y_valid), dtype=np.float32)
    model = LGBMRegressor(**params)
    model.fit(x_train, y_train,eval_set=[(x_valid, y_valid)], early_stopping_rounds=100, verbose=100)
    oof_pred = model.predict(x_valid)
    score = mean_absolute_error(oof_pred, y_valid)
    print('MAE Score:', score)
    return oof_pred, model, score

In [ ]:
oof1, model1, score1 = lgbm(x_train, y_train['target1'], x_valid, y_valid['target1'], params1)
oof2, model2, score2 = lgbm(x_train, y_train['target2'], x_valid, y_valid['target2'], params2)
oof3, model3, score3 = lgbm(x_train, y_train['target3'], x_valid, y_valid['target3'], params3)
oof4, model4, score4 = lgbm(x_train, y_train['target4'], x_valid, y_valid['target4'], params4)
score = (score1+score2+score3+score4) / 4
print(f'MAE Score: {score}')

# 6. Inference

In [ ]:
rosters_cols.remove('date')
scores_cols.remove('date')
standings_cols.remove('date')

In [ ]:
env = mlb.make_env()
iter_test = env.iter_test()

In [ ]:
for (test_df, sample_prediction_df) in iter_test:
    sample_prediction_df = sample_prediction_df.reset_index(drop=True)
    sample_prediction_df['playerId'] = sample_prediction_df['date_playerId'].map(lambda x: int(x.split('_')[1]))

    if test_df['rosters'].iloc[0] == test_df['rosters'].iloc[0]:
        test_rosters = pd.DataFrame(eval(test_df['rosters'].iloc[0]))
    else:
        test_rosters = pd.DataFrame({'playerId': sample_prediction_df['playerId']})
        for col in rosters.columns:
            if col == 'playerId': continue
            test_rosters[col] = np.nan
            
    if test_df['playerBoxScores'].iloc[0] == test_df['playerBoxScores'].iloc[0]:
        test_scores = pd.DataFrame(eval(test_df['playerBoxScores'].iloc[0]))
    else:
        test_scores = pd.DataFrame({'playerId': sample_prediction_df['playerId']})
        for col in scores.columns:
            if col == 'playerId': continue
            test_scores[col] = np.nan
            
    if test_df['standings'].iloc[0] == test_df['standings'].iloc[0]:
        test_standings = pd.DataFrame(eval(test_df['standings'].iloc[0]))
    else:
        test_standings = pd.DataFrame({'playerId': sample_prediction_df['playerId']})
        for col in standings.columns:
            if col == 'playerId': continue
            test_scores[col] = np.nan
                
    test_scores = test_scores.groupby('playerId').sum().reset_index()
    test = sample_prediction_df[['playerId']].copy()
    
    test = test.merge(players['playerId', 'primaryPositionName'], on='playerId', how='left')
    test = test.merge(test_rosters[rosters_cols], on='playerId', how='left')
    test = test.merge(test_scores[scores_cols], on='playerId', how='left')
    test = test.merge(player_target_stats, how='inner', left_on=["playerId"],right_on=["playerId"])
    test = test.merge(awards_count, how = 'left', on = 'playerId')
    test = test.merge(teams, how = 'left', on = 'teamId')
    test['awardId_count'] = test['awardId_count'].fillna(0)
    test = test.merge(playerTwitterFollowers, how = 'left', on ='playerId')
    test = test.merge(teamTwitterFollowers, how = 'left', on ='teamId')
    test = test.merge(test_standings[standings_cols], how = 'left', on = 'teamId')    

    test['label_playerId'] = test['playerId'].map(player2num)
    test['label_primaryPositionName'] = test['primaryPositionName'].map(position2num)
    test['label_teamId'] = test['teamId'].map(teamid2num)
    test['label_status'] = test['status'].map(status2num)
    test['label_leagueId'] = test['leagueId'].map(leagueId2num)
    test['label_divisionId'] = test['divisionId'].map(divisionId2num)
    test_X = test[feature_cols]
    
    pred1 = model1.predict(test_X)
    pred2 = model2.predict(test_X)
    pred3 = model3.predict(test_X)
    pred4 = model4.predict(test_X)
    
    sample_prediction_df['target1'] = np.clip(pred1, 0, 100)
    sample_prediction_df['target2'] = np.clip(pred2, 0, 100)
    sample_prediction_df['target3'] = np.clip(pred3, 0, 100)
    sample_prediction_df['target4'] = np.clip(pred4, 0, 100)
    sample_prediction_df = sample_prediction_df.fillna(0)
    del sample_prediction_df['playerId']
    env.predict(sample_prediction_df)

print('Infrence is Completed')